In [2]:
from dotenv import load_dotenv
import os
import tvdb_v4_official
import pandas as pd
import tqdm

load_dotenv()
tvdb = tvdb_v4_official.TVDB(os.getenv('TVDB_API_KEY'))

# Initial preprocessing

In [ ]:
movies = pd.read_csv(r".\relevant_data\tracking_movies.csv")
movies = movies[movies["entity_type"] == "movie"]
movies = movies[["updated_at", "movie_name", "release_date", "type"]]
print(movies.shape)

In [ ]:
# for each values in the movie_name column, set the release date to the latest release date of the movie
for movie_name in tqdm.tqdm(movies["movie_name"].unique()):
    release_date = movies[movies["movie_name"] == movie_name]["release_date"].max()
    movies.loc[movies["movie_name"] == movie_name, "release_date"] = release_date

for movie_name in tqdm.tqdm(movies["movie_name"].unique()):
    added_date = movies[(movies["movie_name"] == movie_name) & (movies["type"] == "follow")]["updated_at"].min()
    movies.loc[movies["movie_name"] == movie_name, "added_date"] = added_date

movies = movies[movies["type"] == "watch"]
movies = movies.drop(columns=["type"])

In [ ]:
# join with votes
votes = pd.read_csv(r"C:\Users\rik\Desktop\TVAnalysis\relevant_data\ratings_movies.csv")[["movie_name", "vote_key"]]
votes["vote_key"] = votes["vote_key"].apply(lambda x: x.split("-")[-1])
# weird values remapping
votes["vote_key"] = votes["vote_key"].replace({"1": 1, "27": 2, "28": 3, "29": 4, "3": 5})
movies = movies.merge(votes, left_on="movie_name", right_on="movie_name", how="left")
movies["vote_key"] = movies["vote_key"].fillna(0)
movies.head(2)

In [26]:
movies.rename(columns={"movie_name": "title",
                       "updated_at": "seen_date",
                       "vote_key": "stars"}, inplace=True)

In [27]:
movies.to_csv(r".\data_refinement\movies_step0.csv", index=False)
movies.to_excel(r".\data_refinement\movies_step0.xlsx", index=False)

# Get info via API

## Get the tvdb id for each movie by querying via title

In [54]:
movies = pd.read_csv(r".\data_refinement\movies_step0.csv")

In [55]:
id_fixes = {
    "Guardians": [4028, """During the Cold War, a secret organization known as "Patriot" gathers a team of Soviet superheroes, altering and augmenting the DNA of many selected individuals throughout the former state's territory, in order to defend the homeland from superhuman threats. The team includes representatives of the different nationalities of the Soviet Union and allegedly implies restoration of former USSR. Arseniy, codenamed Ursus (Anton Pampushny), can change into a huge, incredibly strong bear-man. Temirkhan, codenamed Khan (Sanjar Madi), has super-speed and wields a pair of curved swords. Lernik, codenamed Ler (Sebastien Sisak), has the ability to control earth and stone. Finally, Xenia (Alina Lanina) has the power of invisibility and can transform her body into water."""],
    "STAND BY ME ドラえもん": [6355, """Noby Nobi is a fourth grader who constantly gets failing grades in his subjects due to his laziness and is always bullied by his classmates Sneech and Big G. His great-great grandson from the 22nd century, Soby, who watches him every day, travels to Noby's timeline while bringing along his robotic cat Doraemon. Soby reveals that if Noby keeps up his act, he will have a disastrous future: he will marry Big G's sister Little G, have his private company burned down, and will be left with a great debt. To circumvent this, he orders Doraemon to help Noby, modifying Doraemon's code to prevent him from returning to the future unless Noby gains a better future."""],
    "Erax": [332587, """Monstrous creatures leap from a magical storybook and unleash mayhem and mischief for Auntie Opal and her tween niece Nina."""],
    "The Thief and the Cobbler": [15517, """The film opens with a narrator describing a prosperous city called the Golden City, ruled by the sleepy King Nod and protected by three golden balls atop its tallest minaret. According to a prophecy, the city would fall to a race of warlike, one-eyed monsters, known as "One-Eyes", should the balls be removed, and could only be saved by "the simplest soul with the smallest and simplest of things". Living in the city are the good-hearted Shoemaking Tack, named for the ubiquitous pair of tacks held in his mouth, and a nameless, unsuccessful yet persistent thief."""],
    "Snowpiercer": [1502, """In 2014, an attempt to counteract global warming through climate engineering backfires catastrophically, and the Earth has entered a new ice age. The only survivors are on the "Snowpiercer," a massive train traveling on a circumnavigational track. Years later, the train has undergone huge economical stratification where the poor endure brutal conditions and the wealthy bask in opulence, which all comes to a head when a young charismatic leader launches a rebellion on board."""],
}

In [56]:
def get_movie_id_and_overview(obj):
    movie_id = obj["tvdb_id"]
    if "overview" in obj.keys():
        overview = obj["overview"]
    elif ("overviews" in obj.keys()) and ("eng" in obj["overviews"].keys()):
        overview = obj["overviews"]["eng"]
    else:
        overview = ""
    return movie_id, overview

In [ ]:
for index, row in tqdm.tqdm(movies.iterrows(), total=movies.shape[0]):
    name = row["title"]
    search_results = tvdb.search(name)
    results_names = [result["name"] for result in search_results]
    equal_names = [n for n in results_names if n == name]
    # manual fixes
    if name in id_fixes.keys():
        movie_id, descripiton = id_fixes[name][0], id_fixes[name][1]
    # when there are multiple names with the same title
    if len(equal_names) == 1:
        name_index = results_names.index(equal_names[0])
        movie_id, description = get_movie_id_and_overview(search_results[name_index])
    # not found
    elif len(search_results) < 1:
        print("No results found for", name)
        movie_id, description = 0, ""
    # just 1 result or no date basis to discriminate
    elif (len(search_results) == 1) or (str(row["release_date"]) == "nan"):
        movie_id, description = get_movie_id_and_overview(search_results[0])
    # multiple results with the same name, but (possibly) different release years
    elif len(search_results) > 1:
        found = False
        year = row["release_date"].split("-")[0]
        for result in search_results:
            if ("first_air_time" in result.keys())\
                and (result["type"] == "movie")\
                and (result["first_air_time"].split("-")[0] == year):
                movie_id, description = get_movie_id_and_overview(result)
                found = True
                break
        if not found:
            movie_id, description = get_movie_id_and_overview(search_results[0])

    movies.loc[index, "tvdb_id"] = movie_id
    movies.loc[index, "description"] = description

movies = movies[movies["tvdb_id"] != 0]

In [ ]:
movies = movies.drop_duplicates()
# see duplicates
movies[movies["tvdb_id"].duplicated(keep=False)].sort_values("tvdb_id")

In [ ]:
# drop duplicates base on tvdb_id, keeping the one with lowest seen_date
movies = movies.sort_values("seen_date")
movies = movies.drop_duplicates("tvdb_id", keep="first")
movies

In [60]:
movies.to_csv(r".\data_refinement\movies_step1.csv", index=False)
movies.to_excel(r".\data_refinement\movies_step1.xlsx", index=False)

## Get the interesting information for each movie

In [61]:
movies = pd.read_csv(r".\data_refinement\movies_step1.csv")

In [ ]:
keys = tvdb.get_movie_extended(31).keys()
keys

In [ ]:
res = []
for index, row in tqdm.tqdm(movies.iterrows(), total=movies.shape[0]):
    movie_id = row["tvdb_id"]
    try:
        data = tvdb.get_movie_extended(movie_id)
    except:
        print("Error in getting details for", movie_id)
        data = {k: "" for k in keys}
    res.append(data)
res = pd.DataFrame(res)

In [ ]:
movies["tvdb_id"] = movies["tvdb_id"].fillna("0").astype(int)
movies = movies.merge(res, left_on="tvdb_id", right_on="id", how="left")
movies = movies[movies["name"].notna()]
movies.shape

In [68]:
movies.to_csv(r"./data_refinement/movies_step2.csv", index=False)
movies.to_excel(r"./data_refinement/movies_step2.xlsx", index=False)

# Power BI formatting

In [125]:
movies = pd.read_csv(r"./data_refinement/movies_step2.csv")
movies.head(2)

,seen_date,title,release_date,added_date,stars,tvdb_id,description,id,name,slug,...,studios,awards,tagOptions,lists,contentRatings,companies,production_countries,inspirations,spoken_languages,first_release
0,2019-11-04 19:12:07,Interstellar,2014-11-05 00:00:00,2019-11-04 19:11:59,5.0,131079,With humanity teetering on the brink of extinc...,131079,Interstellar,interstellar,...,"[{'id': 29, 'name': 'Warner Bros. Pictures', '...","[{'id': 132941, 'year': '2015', 'details': Non...",NaN,"[{'id': 7540, 'name': 'Nolan', 'overview': '.....","[{'id': 252, 'name': 'SAM13', 'country': 'arg'...","{'studio': [{'id': 2063, 'name': 'Paramount Pi...","[{'id': 261788, 'country': 'gbr', 'name': 'Gre...",[],['eng'],"{'country': 'global', 'date': '2014-11-05', 'd..."
1,2019-11-04 19:12:14,Avengers: Endgame,2019-04-24 00:00:00,2019-11-04 19:11:52,4.0,148,The epic conclusion to the Infinity Saga that ...,148,Avengers: Endgame,avengers-endgame,...,"[{'id': 28, 'name': 'Walt Disney Studios', 'pa...","[{'id': 138626, 'year': '2019', 'details': Non...","[{'id': 178, 'tag': 10, 'tagName': 'Relationsh...","[{'id': 4, 'name': 'Marvel Cinematic Universe'...","[{'id': 279, 'name': '12', 'country': 'bra', '...","{'studio': [{'id': 2075, 'name': 'Walt Disney ...","[{'id': 272327, 'country': 'usa', 'name': 'Uni...",[],"['eng', 'jpn', 'xho', 'deu']","{'country': 'grc', 'date': '2019-04-24', 'deta..."


In [126]:
for col in ["stars", "runtime", "budget", "boxOffice"]:
    movies[col] = movies[col].fillna(0).astype(int)
cols_to_drop = ["tvdb_id", "nameTranslations", "overviewTranslations", "score", "status", "lastUpdated", "releases", "remoteIds", "boxOfficeUS", "audioLanguages", "subtitleLanguages", "lists"]
movies = movies.drop(columns=cols_to_drop)

In [ ]:
col_to_explode = ["aliases", "trailers", "genres", "artworks", "characters", "studios", "awards", "tagOptions", "contentRatings", "companies", "production_countries", "inspirations", "spoken_languages"]

In [ ]:
for col in col_to_explode:
    movies[col] = movies[col].apply(lambda x: [y["name"] for y in eval(x)])

In [ ]:
movies = pd.read_csv(r"./data_refinement/movies_step3_power_bi.csv")

# Format data extracting only interesting aspects

In [ ]:
# read again
movies = pd.read_csv(r"./data_refinement/movies_step2.csv")
movies.head(3)

In [236]:
# genres
movies["genres"] = movies["genres"].fillna("[]").apply(lambda x: [g["slug"] for g in eval(x)])

In [237]:
# awards
movies["awards_won"] = movies["awards"].fillna("[]").apply(lambda x: eval(x)).apply(lambda x: [g["name"] + " - " + g["category"] for g in x if g["isWinner"]])
movies["awards_candidate"] = movies["awards"].fillna("[]").apply(lambda x: eval(x)).apply(lambda x: [g["name"] + " - " + g["category"] for g in x])
movies["n_awards_won"] = movies["awards_won"].apply(lambda x: len(x))
movies["n_awards_candidate"] = movies["awards_candidate"].apply(lambda x: len(x))
movies = movies.drop(columns=["awards"])

In [ ]:
# tags
tag_types = movies["tagOptions"].fillna("[]").apply(lambda x: eval(x)).apply(lambda x: [g["tagName"] for g in x]).explode()
tag_types = tag_types.unique().tolist()
# remove nan
tag_types = [x for x in tag_types if str(x) != 'nan']
for tag_type in tag_types:
    print(tag_type)
    movies[tag_type.lower().replace(" ", "_")] = movies["tagOptions"].fillna("[]").apply(lambda x: eval(x)).apply(lambda x: [g["name"] for g in x if g["tagName"] == tag_type])
movies = movies.drop(columns=["tagOptions"])

In [ ]:
movies["contentRatings"] = movies["contentRatings"].fillna("[]").apply(lambda x: eval(x)).apply(lambda x: [g["name"] for g in x if g["country"] == "usa"])
movies["contentRatings"] = movies["contentRatings"].apply(lambda x: x[0] if len(x) > 0 else "").apply(lambda x: x.replace("TV-", ""))
movies["contentRatings"] = movies["contentRatings"].replace({"MA": "NC-17"})
# replace all values not in ["G", "PG", "PG-13", "R", "NC-17"] with ""
movies["contentRatings"] = movies["contentRatings"].apply(lambda x: x if x in ["G", "PG", "PG-13", "R", "NC-17"] else "")
movies["contentRatings"].value_counts()

In [ ]:
studio = movies["companies"].fillna("[]").apply(lambda x: eval(x)).apply(lambda x: x["studio"])
studio = studio.apply(lambda x: x[0]["slug"] if len(x) > 0 else "")
movies = movies.drop(columns=["companies"])
movies["studio"] = studio
movies["studio"].value_counts()

In [ ]:
# characters
roles = movies["characters"].fillna("[]").apply(lambda x: eval(x))
roles = roles.apply(lambda x: [g["peopleType"] for g in x])
# unique and remove nan
roles = roles.explode().unique().tolist()
roles = [x for x in roles if str(x) != 'nan']
movies["actor"] = ""
movies["actors_not_featured"] = ""

for role in roles:
    print(role)
    movies[role.lower().replace(" ", "_")] = movies["characters"].fillna("[]").apply(lambda x: eval(x)).apply(lambda x: [g["personName"] for g in x if g["peopleType"] == role])

for i, row in movies.iterrows():
    actors = row["characters"]
    if str(actors) == "nan":
        actors = "[]"
    actors = eval(actors)
    real_actors = [a["personName"] for a in actors if (a["peopleType"] == "Actor") and (a["isFeatured"])]
    voice_actors = [a["personName"] for a in actors if (a["peopleType"] == "Actor") and (not a["isFeatured"])]
    movies.at[i, "actor"] = real_actors
    movies.at[i, "actors_not_featured"] = voice_actors

movies = movies.drop(columns=["characters"])

In [243]:
# fix column names for ease of use
remapped_columns = {
    "updated_at": "seen_date",
    "slug": "title",
    "tvdb_id": "id",
    "vote_key": "stars",
}
col_to_remove = ["score", "studios", "creator", "musical_guest", "crew", "executive_producer"]
movies = movies.rename(columns=remapped_columns)
movies = movies.drop(columns=col_to_remove)
movies = movies.drop_duplicates(["title"])

In [244]:
movies.to_csv(r"./movies_step2.csv", index=False)
movies.to_excel(r"./movies_step2.xlsx", index=False)
movies.to_json(r"./movies_step2.json", orient="records", lines=True)

# step 2 cleaning

In [197]:
movies = pd.read_csv(r"./movies_step2.csv")
movies["writer"] = movies["writer"].apply(lambda x: eval(x))
movies["is_short"] = movies["movie_type_or_format"].apply(lambda x: "Short" in eval(x))
# delete movie_type_or_format
movies = movies.drop(columns=["movie_type_or_format"])

In [198]:
def remove_useless_genres(x, subgenre):
    if len(x) > 1:
        if subgenre in x:
            x.remove(subgenre)
    return x

movies["sub-genre"] = movies["sub-genre"].fillna("[]").apply(lambda x: eval(x)).apply(lambda x: remove_useless_genres(x, "Adaptation")).fillna("")
movies["sub-genre"] = movies["sub-genre"].apply(lambda x: remove_useless_genres(x, "Blockbuster")).fillna("")
movies["sub-genre"] = movies["sub-genre"].apply(lambda x: remove_useless_genres(x, "Cult")).fillna("")
movies["sub-genre"] = movies["sub-genre"].apply(lambda x: remove_useless_genres(x, "Nature")).fillna("")
movies["sub-genre"] = movies["sub-genre"].apply(lambda x: remove_useless_genres(x, "Period")).fillna("")
movies["sub-genre"] = movies["sub-genre"].apply(lambda x: remove_useless_genres(x, "Critically Acclaimed")).fillna("")
movies["sub-genre"] = movies["sub-genre"].apply(lambda x: x[0] if len(x) > 0 else "").fillna("")

In [199]:
# take just the first element from columns geographic_location, social_topics, time_period, mood_or_tone, credit_scenes
movies["geographic_location"] = movies["geographic_location"].fillna("[]").apply(lambda x: eval(x)).apply(lambda x: x[0] if len(x) > 0 else "")
movies["social_topics"] = movies["social_topics"].fillna("[]").apply(lambda x: eval(x)).apply(lambda x: x[0] if len(x) > 0 else "")
movies["time_period"] = movies["time_period"].fillna("[]").apply(lambda x: eval(x)).apply(lambda x: x[0] if len(x) > 0 else "")
movies["mood_or_tone"] = movies["mood_or_tone"].fillna("[]").apply(lambda x: eval(x)).apply(lambda x: x[0] if len(x) > 0 else "")
movies["credit_scenes"] = movies["credit_scenes"].fillna("[]").apply(lambda x: eval(x)).apply(lambda x: x[0] if len(x) > 0 else "")
# onehot encode plot_characteristics, relationship_types

In [200]:
movies.to_csv(r"./movies_step3.csv", index=False)
movies.to_excel(r"./movies_step3.xlsx", index=False)

# value normalization after exploration

In [224]:
movies = pd.read_csv(r"./movies_step3.csv")

In [ ]:
movies["studio"].unique()

In [229]:
movies["studio"] = movies["studio"].replace({
    "netflix-studios": "netflix",
    "20th-century-fox-animation": "20th-century-fox",
    "colombia-pictures": "columbia-pictures",
    "dreamworks-animation": "dreamworks",
    "lions-gate-films": "lionsgate",
    "metro-goldwyn-mayer": "mgm",
    "samuel-goldwyn-films": "mgm",
    "paramount-players": "paramount",
    "sony-pictures-classics": "sony",
    "sony-pictures-entertainment": "sony",
    "tristar-pictures": "tristar",
    "tristar-productions": "tristar",
    "walt-disney-animation": "disney",
    "walt-disney-animation-studios": "disney",
    "walt-disney-studios": "disney",
    "walt-disney-studios-motion-pictures": "disney",
    "disney-plus": "disney",
    "warner-bros-animation": "warner-bros",
    "warner-bros-pictures": "warner-bros",
})

In [ ]:
movies["studio"].value_counts().head(40)

In [217]:
movies.to_csv(r"./movies_step4.csv", index=False)
movies.to_excel(r"./movies_step4.xlsx", index=False)

In [231]:
movies.to_json(r"./movies_step4.json", orient="records")